In [1]:
# run in docker:
# sudo docker run -p 8890:8888 -v /home/tehdog/data/dev/2019/pic2kcal-cv-praktikum:/tf/notebooks --runtime=nvidia -it --rm tensorflow/tensorflow:1.12.0-gpu-py3 jupyter notebook --allow-root --notebook-dir=/tf/notebooks
# crashes in TF1.4!
# does not work with Arch Linux TF installation!

In [2]:
%cd ..

/tf/notebooks


In [3]:
!pip install tensorflow-hub
!pip install sentencepiece
!pip install tf-sentencepiece
!pip install tqdm

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

# Set up graph.
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  en_de_embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling/en-de/1")
  embedded_text = en_de_embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0701 09:44:39.742711 139950204315392 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [5]:
from pathlib import Path
import json
import re
import random
import heapq
from operator import itemgetter
from tqdm import tqdm_notebook as tqdm
from math import ceil

data_dir = Path("data")

In [6]:
# jq '.[]' fddb_data_v3.json > fddb_data_v3.jsonl
# jq 'select(([.Bilder[]|select(.title != "Noch kein Foto vorhanden.")]|length) > 0)' fddb_data_v3.jsonl | jq -s > fddb_data_v3_withimg.json
with open(str(data_dir / "fddb_data_v3_withimg.json"), encoding='utf-8') as f:
    fddb = json.load(f)
    # todo: make unique here
    _out_names = [e["name"] for e in fddb]

# jq '.ingredients[]|select(.ingredient)|.ingredient' processed_data.jsonl | jq -s unique > ingredients.json
with open(str(data_dir / "recipes/ingredients.json"), encoding='utf-8') as f:
    _in_names = json.load(f)


In [23]:
def normalize_ingredient(ing: str):
    ing = re.sub(r"\(([^)])\)", "\g<1>", ing)  # remove stuff in parens
    ing = re.sub(r"\([^)]+\)", "", ing)  # remove stuff in parens
    ing = re.sub(r"(\d+,)?\d+ k?g\b", "", ing)  # remove xyz gram
    ing = re.sub(r",.*", "", ing)
    ing = re.sub(r"\bzum .*", "", ing)
    ing = re.sub(r"\boder\b.*", "", ing)
    ing = ing.strip()
    return ing

from extract_ingredients.util import normalize_out_ingredient
    

In [8]:
in_names = list({normalize_ingredient(ing) for ing in _in_names})
print(len(_in_names))
in_names.sort()
len(in_names)

258944


21444

In [9]:
print(len(_out_names))
out_names = list({ning for ing in _out_names for ning in normalize_out_ingredient(ing)})
out_names.sort()
len(out_names)

122977


108084

In [10]:
#def Matcher():
#    def __init__(self, data_left, data_right, preproc_left, preproc_right):
#        pre_left = list(map(preproc_left, data_left))
#        self.left_vecs = list(zip(data_left, pre_left, get_sentence_vectors(pre_left)))
#        
#        pre_right = list(map(preproc_right, data_right))
#        self.right_vecs = list(zip(data_right, pre_right, get_sentence_vectors(pre_right)))

In [11]:
def make_chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

In [12]:
def get_sentence_vectors(texts):
    bs = 10000
    ccount = len(texts)//bs
    chunks = make_chunks(texts, bs)
    if ccount >= 3:
        chunks = tqdm(chunks, total=ccount)
    for chunk in chunks:
        yield from session.run(embedded_text, feed_dict={text_input: chunk})

In [13]:
def get_match(search: np.array, out_vecs, limit=10):
    it = ((v[0], np.dot(v[1], search)) for v in out_vecs)
    res_list = heapq.nlargest(limit, it, key=itemgetter(1))
    return res_list

In [14]:
out_vecs = list(zip(out_names, get_sentence_vectors(out_names)))

In [15]:
out_vecs[0][1].shape

(512,)

In [16]:
in_names[0], out_names[0]

('8 Kräuter', '"DIE FEINE" Geflügel-Fleischwurst, würzi...')

In [17]:
in_vecs = list(zip(in_names, get_sentence_vectors(in_names)))

In [18]:
for ingredient, vec in random.sample(in_vecs, 100):
    res_list = get_match(vec, out_vecs)
    print("{} -> {}".format(ingredient, res_list[0]))

Kräutermeersalz und Pfeffer -> ('Kräutersalz mit Meersalz', 0.8910994)
Rosenblätter einige frische -> ('frische Lasagne-Blätter', 0.78550977)
Butterschmalz für die Form -> ('Butterschmalz', 0.8332726)
Erbsen . TK -> ('Erbsen, fein TK', 0.8578725)
Hafer - Kleie -> ('Hafer Kleie', 0.93807006)
Muffinförmchen -> ('Gebäckröllchen', 0.8753226)
Äpfel  für die Garnitur -> ('Püree aus Äpfeln', 0.8080305)
Rote Bete -> ('Rote Bete', 0.99999994)
Tortenboden mit Rand -> ('Feiner Biskuit Tortenboden', 0.76696867)
Orangenaroma -> ('Orangenlimonade', 0.8714565)
Blumenkohl in Röschen -> ('Blumenkohl Röschen', 0.9352437)
Zitronenmelisse zur Dekoration -> ('Zitronenmelisse', 0.79763734)
Gartenkräuter  nach Belieben -> ('Gartenkräuter', 0.89898866)
Garnelen  Gr. 16/20 -> ('Penne Rigate N. 19', 0.7066887)
Puddingpulver für Grießpudding -> ('Grießpudding pur', 0.8798977)
Eiweißbrot -> ('Eiweißbrot', 0.9999999)
Erythrit  light -> ('Erythrit', 0.8693255)
Joghurt Vanillegeschmack -> ('Joghurt Vanille', 0.94737

In [19]:
z = [inv for inv in in_vecs if inv[0] == 'Kartoffeln'][0]
get_match(z[1], out_vecs)

[('Kartoffeln', 1.0),
 ('Kartoffelstock', 0.8959534),
 ('Kartoffeln roh', 0.8905239),
 ('Kartoffeln gebacken', 0.8904908),
 ('Kartoffelscheiben', 0.8901619),
 ('Kartoffeln, ganz', 0.88885343),
 ('Kartoffel', 0.88085693),
 ('Kartoffelnudeln', 0.8792579),
 ('Gekochte Kartoffeln', 0.8718474),
 ('geschälte Kartoffeln', 0.8708629)]

In [20]:
z = [inv for inv in in_vecs if inv[0] == 'Zwiebeln'][0]
get_match(z[1], out_vecs, limit=10)

[('Zwiebeln', 1.0),
 ('Zwiebel', 0.9583577),
 ('Zwiebelringe', 0.9388668),
 ('Zwiebelmett', 0.9328606),
 ('Zwiebelsenf', 0.9232825),
 ('Zwiebelschmalz', 0.91526103),
 ('Rote Zwiebeln', 0.8996499),
 ('Zwiebeln, gegart', 0.8959136),
 ('Rstzwiebeln', 0.89036834),
 ('Zwiebelfleisch', 0.88157207)]

In [24]:
# i thought multiprocessing is impossible in jupyter notebook?
# better not question it
from multiprocessing import Pool

def jsonable(e):
    return [(a,float(b)) for a, b in e]

def get_match_w(t):
    ing, vec = t
    return ing, get_match(vec, out_vecs)

all_ings = {}
with Pool(8) as pool:
    for ing, match in pool.imap(get_match_w, tqdm(in_vecs), chunksize=100):
        all_ings[ing] = jsonable(match)

In [25]:

all_ings_orig = {ing: all_ings[normalize_ingredient(ing)] for ing in _in_names}

In [26]:
with open("data/recipe-ingredient-to-fooddb.json", "w") as f:
    json.dump(all_ings_orig, f)